In [1]:
from spacerocks import MPCHandler
from spacerocks.time import Time
from spacerocks.nbody import Simulation, Force, Integrator
from spacerocks.spice import SpiceKernel
from spacerocks import SpaceRock
from spacerocks.transforms import calc_true_anomaly_from_mean_anomaly
from spacerocks import RockCollection

import numpy as np

In [2]:
kernel = SpiceKernel.defaults()


Using default configuration:
  Kernel paths: ["/Users/kjnapier/.spacerocks/spice"]
  Download directory: "/Users/kjnapier/.spacerocks/spice"
  Auto-download: true

Processing kernel: latest_leapseconds.tls
✓ Found existing kernel at: /Users/kjnapier/.spacerocks/spice/latest_leapseconds.tls
Loading kernel: /Users/kjnapier/.spacerocks/spice/latest_leapseconds.tls

Processing kernel: de440s.bsp
✓ Found existing kernel at: /Users/kjnapier/.spacerocks/spice/de440s.bsp
Loading kernel: /Users/kjnapier/.spacerocks/spice/de440s.bsp

Processing kernel: earth_1962_240827_2124_combined.bpc
✓ Found existing kernel at: /Users/kjnapier/.spacerocks/spice/earth_1962_240827_2124_combined.bpc
Loading kernel: /Users/kjnapier/.spacerocks/spice/earth_1962_240827_2124_combined.bpc


In [3]:
# Create a rockcollection with the MPC data
handler = MPCHandler()
nea_collection = handler.fetch_data(
    catalog = "nea_extended",
    output_format = "dataframe"
)

print(f"Loaded {len(nea_collection)} NEAs")


Using existing feather file: /Users/kjnapier/.spacerocks/mpc/nea_extended.feather
Loaded 37460 NEAs


In [4]:
nea_collection

,H,G,Epoch,M,Peri,Node,i,e,a,Principal_desig,orbit_type
0,10.41,0.15,2460600.5,86.66754,178.91030,304.27434,10.82773,0.222691,1.458181,A898 PA,Amor
1,15.59,0.15,2460600.5,148.45068,156.21553,183.85715,11.57526,0.546779,2.636157,A911 TB,Amor
2,13.79,0.15,2460600.5,340.19843,350.47423,110.42302,9.39880,0.571093,2.472525,A918 AA,Amor
3,9.18,0.15,2460600.5,6.98496,132.49616,215.49497,26.68673,0.532826,2.665299,A924 UB,Amor
4,17.37,0.15,2460600.5,271.70081,26.71763,171.26079,11.86848,0.434718,1.920154,1932 EA1,Amor
...,...,...,...,...,...,...,...,...,...,...,...
37455,19.88,0.15,2460600.5,16.79916,142.73350,155.01613,6.40102,0.397912,2.136127,2021 NF1,Amor
37456,18.43,0.15,2460600.5,261.16611,336.66141,30.64476,10.57609,0.597588,2.632801,2021 NS5,Amor
37457,18.93,0.15,2460600.5,220.33161,344.71167,265.00727,25.96643,0.511470,1.686187,2023 PB,Apollo
37458,18.96,0.15,2460600.5,118.61192,233.96853,153.80370,38.47990,0.368584,1.961853,2023 QN,Amor


In [5]:
collection = RockCollection()
# Add all elements in nea_collection to rock_collection one row at a time
for i, row in nea_collection.iterrows():
    # Create a rock object
    rock = SpaceRock.from_kepler(name = row["Principal_desig"],
                                 q = row["a"]* (1 - row["e"]),
                                 e = row["e"],
                                 inc = np.radians(row["i"]),
                                 arg = np.radians(row["Peri"]),
                                 node = np.radians(row["Node"]),
                                 true_anomaly = np.radians(calc_true_anomaly_from_mean_anomaly(row["M"], row["e"])),
                                 epoch = Time(row["Epoch"], "utc", "jd"),
                                 reference_plane = "ECLIPJ2000",
                                 origin = "Sun")
    # Add the rock object to the rock_collection
    collection.add(rock)

In [6]:
collection

RockCollection: 37460 rocks

In [7]:
# Create simulation with planets
epoch = Time.now()
sim = Simulation.planets(epoch=epoch, reference_plane="ECLIPJ2000", origin="Sun")

# Add forces for better accuracy
sim.add_force(Force.solar_gr())  # General relativity
sim.add_force(Force.solar_j2())  # Solar oblateness

# Set up integrator
sim.set_integrator(Integrator.ias15(timestep = 10)) 

In [8]:
collection.analytic_propagate(epoch)

In [10]:
for rock in collection:
    sim.add(rock)
    

print("Added NEAs to simulation")

Added NEAs to simulation


In [10]:
# Get Earth's position at each step and calculate distances
earth = sim.get_particle("earth barycenter")
initial_positions = np.array([rock.position for rock in collection])

In [ ]:
# Check positions every 10 days
check_points = np.arange(0, 3650.25, 10)
close_approaches = []

sim.set_epoch(epoch)  # Reset to start

for days in check_points:
    current_epoch = epoch + days
    sim.integrate(current_epoch)
    
    # Get positions
    earth_pos = sim.get_particle("earth barycenter").position
    asteroid_positions = np.array([rock.position for rock in collection])
    
    # Calculate distances
    distances = np.linalg.norm(asteroid_positions - earth_pos, axis=1)
    
    # Record close approaches (less than 0.1 AU)
    for i, dist in enumerate(distances):
        if dist < 0.05:  # 0.1 AU threshold
            close_approaches.append({
                'asteroid': collection[i].name,
                'date': current_epoch.calendar(),
                'distance': dist
            })

# Print results
print("\nDetected Close Approaches (<0.1 AU):")
for approach in close_approaches:
    print(f"{approach['asteroid']}: {approach['distance']:.3f} AU on {approach['date']}")
    
    
    

In [83]:
len(close_approaches)

15474